In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib

In [4]:
df = pd.read_csv('../../data/training_data.csv',sep=',')
df.drop(columns=['Complexity Average','Rating Average'],inplace=True)
df

,Min Players,Max Players,Play Time,Min Age,Users Rated,Owned Users,Abstract Games,Children's Games,Customizable Games,Family Games,...,Wargames,Social Interaction,Strategy and Planning,Conflict Resolution,Exploration,Randomness and Luck,Resource Management,Puzzle Solving,Miscellaneous / Other,Collaboration
0,1,4,120.000000,14,42055,68323.0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
1,2,4,60.000000,13,41643,65294.0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
2,2,4,120.000000,14,19217,28785.0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,5,120.000000,12,64864,87099.0,0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
4,3,6,431.858995,14,13468,16831.0,0,0,0,0,...,0,1,1,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20315,2,2,30.000000,4,1340,427.0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
20316,2,99,60.000000,5,2154,1533.0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
20317,2,4,30.000000,3,4006,5788.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
20318,2,6,30.000000,3,3783,4400.0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
X = df.drop(columns=['Owned Users'])
y = df['Owned Users']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()), 
    ('xgb', xgb.XGBRegressor(random_state=42,objective='reg:squarederror'))  
])

In [6]:
param_grid_xgb = {
    'xgb__n_estimators': [100, 200, 300],  
    'xgb__learning_rate': [0.01, 0.05, 0.1], 
    'xgb__max_depth': [3, 5, 7],  
    'xgb__subsample': [0.8, 1.0], 
    'xgb__colsample_bytree': [0.8, 1.0],  
    'xgb__min_child_weight': [1, 3, 5]  
}

grid_search = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best hyperparameters: {'xgb__colsample_bytree': 0.8, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__min_child_weight': 1, 'xgb__n_estimators': 100, 'xgb__subsample': 1.0}


In [12]:
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

Mean Squared Error: 0.511070474906194
Mean Absolute Error: 0.5298514097279445
R² Score: 0.3915800205390555


In [13]:
joblib.dump(grid_search.best_estimator_, '../../models/rating_boost_regression.pkl')

['../../models/rating_boost_regression.pkl']